In [37]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.utils import to_categorical


def extract_age(filename):
    # Remove the file extension
    filename = os.path.splitext(filename)[0]

    # Split the filename into parts
    parts = filename.split('_')

    # Extract the birth date and photo year
    birth_date = parts[-2]  # Assuming birth date is the second last part
    photo_year = parts[-1]  # Assuming photo year is the last part

    # Extract the year from the birth date and convert to integer
    birth_year = int(birth_date.split('-')[0])

    # Convert the photo year to integer
    photo_year = int(photo_year)

    # Calculate age
    age = photo_year - birth_year

    age = max(0, age)
    age = min(100, age)

    return age


def load_data(path):
    faces = []
    ages = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.jpg'):
                image_path = os.path.join(root, file)
                image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
                image = cv.resize(image, (60, 60))
                faces.append(image)
                age = extract_age(file)
                ages.append(age)
    return np.array(faces), np.array(ages)

path = 'C:\\Users\\mevin\\Downloads\\imdb_crop'
faces, ages = load_data(path)
faces = faces / 255.0
faces = np.expand_dims(faces, axis=-1)  # Add an extra dimension for the channels

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(60, 60, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(101, activation='softmax')  # Assuming age range 0-100
])

model.compile(
    optimizer=Adam(),
    loss=SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

model.fit(
    faces,
    ages,
    batch_size=32,
    epochs=50,
    validation_split=0.2
)

model.summary()

model.save('age_recognition.h5')
if os.path.isfile('age_recognition.h5'):
    print("Model saved successfully!")
else:
    print("Model did not save correctly.")

Epoch 1/50
11519/11519 [==============================] - 413s 36ms/step - loss: 3.9464 - accuracy: 0.0362 - val_loss: 3.8957 - val_accuracy: 0.0398
Epoch 2/50
11519/11519 [==============================] - 400s 35ms/step - loss: 3.8951 - accuracy: 0.0403 - val_loss: 3.8731 - val_accuracy: 0.0397
Epoch 3/50
11519/11519 [==============================] - 392s 34ms/step - loss: 3.8707 - accuracy: 0.0429 - val_loss: 3.8624 - val_accuracy: 0.0424
Epoch 4/50
11519/11519 [==============================] - 396s 34ms/step - loss: 3.8547 - accuracy: 0.0453 - val_loss: 3.8576 - val_accuracy: 0.0419
Epoch 5/50
11519/11519 [==============================] - 395s 34ms/step - loss: 3.8415 - accuracy: 0.0466 - val_loss: 3.8647 - val_accuracy: 0.0414
Epoch 6/50
11519/11519 [==============================] - 432s 38ms/step - loss: 3.8303 - accuracy: 0.0483 - val_loss: 3.8642 - val_accuracy: 0.0409
Epoch 7/50
11519/11519 [==============================] - 403s 35ms/step - loss: 3.8196 - accuracy: 0.0496

c:\Users\mevin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved successfully!


In [1]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from sklearn.model_selection import train_test_split

def extract_age(filename):
    # Remove the file extension
    filename = os.path.splitext(filename)[0]

    # Split the filename into parts
    parts = filename.split('_')

    # Extract the birth date and photo year
    birth_date = parts[-2]  # Assuming birth date is the second last part
    photo_year = parts[-1]  # Assuming photo year is the last part

    # Extract the year from the birth date and convert to integer
    birth_year = int(birth_date.split('-')[0])

    # Convert the photo year to integer
    photo_year = int(photo_year)

    # Calculate age
    age = photo_year - birth_year

    age = max(0, age)
    age = min(100, age)

    return age
def load_data(path):
    faces = []
    ages = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.jpg'):
                image_path = os.path.join(root, file)
                image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
                image = cv.resize(image, (60, 60))
                faces.append(image)
                age = extract_age(file)
                ages.append(age)
    return np.array(faces), np.array(ages)

path = 'C:\\Users\\mevin\\Downloads\\imdb_crop'
faces, ages = load_data(path)
faces = faces / 255.0
faces = np.expand_dims(faces, axis=-1)

# Split the data into training, validation, and testing sets
faces_train_val, faces_test, ages_train_val, ages_test = train_test_split(faces, ages, test_size=0.15, random_state=42)
faces_train, faces_val, ages_train, ages_val = train_test_split(faces_train_val, ages_train_val, test_size=0.176, random_state=42)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(60, 60, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(101, activation='softmax')
])

model.compile(
    optimizer=Adam(),
    loss=SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

model.fit(
    faces_train,
    ages_train,
    batch_size=32,
    epochs=50,
    validation_data=(faces_val, ages_val)
)

model.summary()

model.save('age_recog.keras')
if os.path.isfile('age_recog.keras'):
    print("Model saved successfully!")
else:
    print("Model did not save correctly.")


KeyboardInterrupt: 